# Tutorial 2 - Classification (Test)

## Open Dataset : MURA Dataset (Ver 1.0)

- MUsculoskeletal RAdiographs (Bone X-rays)         
- https://stanfordmlgroup.github.io/competitions/mura/
- https://arxiv.org/abs/1712.06957

In [1]:
# Python 3.6
from __future__ import absolute_import, division, print_function
import os
import numpy as np
import h5py
import matplotlib.pyplot as plt

import keras
from keras.layers import Input

import keras.applications.inception_v3 as inception_v3
from keras.models import Sequential, load_model
from keras.callbacks import  ModelCheckpoint, CSVLogger, ReduceLROnPlateau
import tensorflow as tf


Using TensorFlow backend.


## Parameter Setting

In [2]:
########################################################################
########################################################################
save_date = '1022'
save_csv = 'batch.csv'
model_name = 'MURA_Inception_v3_1022_batch1.h5'
########################################################################
########################################################################

## Path Setting

In [3]:
datapath = '../Dataset/MURA-v1.0/h5py/batch/'


save_path = os.path.join(os.getcwd(), 'Result/saved_model')
if not os.path.isdir(save_path):
    os.makedirs(save_path)    


tf_graph_path = './Result/tf_graph/'    
if not os.path.isdir(tf_graph_path):
    os.makedirs(tf_graph_path)   
    
graph_path = './Result/graph_log/'    
if not os.path.isdir(graph_path):
    os.makedirs(graph_path)   

## Load Test Dataset

In [4]:
val = h5py.File(datapath +'Validation.h5', 'r')
X_test = val['Image']
X_test = np.array(X_test)
y_test = val['Label']
y_test = np.array(y_test)

print(np.shape(X_test))
print(np.shape(y_test))

(7428, 299, 299, 3)
(7428, 7)


## Load Trained Model

In [5]:
#####################################################################################
#####################################################################################
trained_model = load_model('./Result/saved_model/' + model_name)
#####################################################################################
#####################################################################################

## Prediction

In [14]:
from sklearn import metrics
from sklearn.metrics import accuracy_score
    
print("*** Predictction ***")
y_pred_temp = trained_model.predict(X_test, verbose=0)

y_pred = []
y_pred_prob0 =[]
y_pred_prob1 =[]
y_pred_prob2 =[]
y_pred_prob3 =[]
y_pred_prob4 =[]
y_pred_prob5 =[]
y_pred_prob6 =[]


for i in range(len(y_pred_temp)):
    y_pred.append(np.argmax(y_pred_temp[i], axis=None, out=None))
    y_pred_prob0.append(round(y_pred_temp[i,0], 3))
    y_pred_prob1.append(round(y_pred_temp[i,1], 3))
    y_pred_prob2.append(round(y_pred_temp[i,2], 3))
    y_pred_prob3.append(round(y_pred_temp[i,3], 3))
    y_pred_prob4.append(round(y_pred_temp[i,4], 3))
    y_pred_prob5.append(round(y_pred_temp[i,5], 3))
    y_pred_prob6.append(round(y_pred_temp[i,6], 3))


### One-hot decoding
y_test_mod = []
for i in range(len(y_test)):
    if y_test[i][0] == 1:
        y_test_mod.append(0)
    elif y_test[i][1] == 1:
        y_test_mod.append(1)
    elif y_test[i][2] == 1:
        y_test_mod.append(2)
    elif y_test[i][3] == 1:
        y_test_mod.append(3)        
    elif y_test[i][4] == 1:
        y_test_mod.append(4)        
    elif y_test[i][5] == 1:
        y_test_mod.append(5)                
    elif y_test[i][6] == 1:
        y_test_mod.append(6)                        

print("\n***** Prediction Complete *****")

*** Predictction ***

***** Prediction Complete *****


## Evaluation

In [15]:
""" Confusion Matrix """
confusion = metrics.confusion_matrix(y_test_mod, y_pred)
print(confusion)

TP = confusion[1, 1]
TN = confusion[0, 0]
FP = confusion[0, 1]
FN = confusion[1, 0]

""" Accuracy """
print("Accuracy: ", round(((TP + TN)/(TP + TN + FP + FN))*100, 2))

""" Sensitivity """
sensitivity = round((TP/(FN+TP))*100, 2)
print("Sensitivity: ", sensitivity)

""" Specificity """
specificity = round((TN/(TN+FP))*100, 2)
print("Specificity: ", specificity)

""" PPV """
PPV = round((TP/(TP+FP))*100, 2)
print("PPV: ", PPV)

""" NPV """
NPV = round((TN/(TN+FN))*100, 2)
print("NPV: ", NPV)

""" AUC """
pass

[[ 917    1   20    2   29   28    5]
 [   2  979    3   23    5    5    6]
 [   6    6  334    2   19    2    3]
 [   1   39    0 1048    0    2   19]
 [   9    2   10    0  223   24    0]
 [  10    0    1    0   20 1659    2]
 [   3   23   31   29    6   17 1853]]
Accuracy:  99.84
Sensitivity:  99.8
Specificity:  99.89
PPV:  99.9
NPV:  99.78


## Save Result (.csv)

In [22]:
from itertools import zip_longest
import csv 

###################################################################
##################################################################
lists = [y_pred_prob0, y_pred_prob1, y_pred_prob2, y_pred_prob3, y_pred_prob4, y_pred_prob5, y_pred_prob6, y_pred, y_test_mod] ### ID 추가!
export_data = zip_longest(*lists, fillvalue = '')
with open('./Result/result_csv/' + save_csv, 'w') as f:
    writer = csv.writer(f)
    writer.writerow(("Prob0", "Prob1", "Prob2", "Prob3", "Prob4", "Prob5", "Prob6", "Pred", "GT")) ###
    writer.writerows(export_data)
###################################################################
###################################################################

## Check Result

In [27]:
import pandas as pd

Test_csv_result = pd.read_csv("./Result/result_csv/batch1.csv")
Test_csv_result.head()

,Prob0,Prob1,Prob2,Prob3,Prob4,Prob5,Prob6,Pred,GT
0,0.043,0.196,0.163,0.298,0.115,0.055,0.130,3,0
1,0.876,0.001,0.115,0.000,0.008,0.000,0.001,0,0
2,0.945,0.000,0.001,0.000,0.005,0.048,0.000,0,0
3,0.323,0.035,0.262,0.009,0.104,0.115,0.151,0,0
4,0.983,0.000,0.012,0.000,0.005,0.000,0.000,0,0


## Homework : ID 추가 / Draw AUC 